In [7]:
import pandas as pd 
import keras
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten

import os
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import cv2
import h5py
import numpy as np
%matplotlib inline

Using TensorFlow backend.


In [9]:
!pip install --upgrade tensorflow

    100% |████████████████████████████████| 49.1MB 990kB/s 
Requirement not upgraded as not directly required: protobuf>=3.4.0 in /usr/local/lib/python3.6/dist-packages (from tensorflow) (3.5.2.post1)
Requirement not upgraded as not directly required: astor>=0.6.0 in /usr/local/lib/python3.6/dist-packages (from tensorflow) (0.6.2)
Requirement not upgraded as not directly required: grpcio>=1.8.6 in /usr/local/lib/python3.6/dist-packages (from tensorflow) (1.11.0)
Requirement not upgraded as not directly required: wheel>=0.26 in /usr/local/lib/python3.6/dist-packages (from tensorflow) (0.31.0)
Requirement not upgraded as not directly required: six>=1.10.0 in /usr/local/lib/python3.6/dist-packages (from tensorflow) (1.11.0)
Requirement not upgraded as not directly required: gast>=0.2.0 in /usr/local/lib/python3.6/dist-packages (from tensorflow) (0.2.0)
    100% |████████████████████████████████| 3.1MB 13.6MB/s 
Requirement not upgraded as not directly required: numpy>=1.13.3 in /usr/local

In [5]:
!pip install -q xlrd
!git clone https://github.com/upasna01/dog-breed-classification-kaggle
!ls dog-breed-classification-kaggle

fatal: destination path 'dog-breed-classification-kaggle' already exists and is not an empty directory.
labels	sample_submission  test  train


In [8]:
dog_train = pd.read_csv('dog-breed-classification-kaggle/labels/labels.csv')
dog_test = pd.read_csv('dog-breed-classification-kaggle/sample_submission/sample_submission.csv')
dog_train.head(10)

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever
5,002211c81b498ef88e1b40b9abf84e1d,bedlington_terrier
6,00290d3e1fdd27226ba27a8ce248ce85,bedlington_terrier
7,002a283a315af96eaea0e28e7163b21b,borzoi
8,003df8b8a8b05244b1d920bb6cf451f9,basenji
9,0042188c895a2f14ef64a918ed9c7b64,scottish_deerhound


In [0]:
targets_series = pd.Series(dog_train['breed'])
breed = pd.get_dummies(targets_series, sparse = True)
breed_labels = np.asarray(breed)
image_size = 224
x_train = []
y_train = []
x_test = []

In [3]:
!pip uninstall tqdm
!pip install --upgrade --no-deps --force-reinstall tqdm

Skipping tqdm as it is not installed.
    100% |████████████████████████████████| 51kB 3.5MB/s 


In [10]:
from tqdm import tqdm
i = 0 
for f, breed in tqdm(dog_train.values):
    img = cv2.imread('dog-breed-classification-kaggle/train/train/{}.jpg'.format(f))
    label = breed_labels[i]
    x_train.append(cv2.resize(img, (image_size, image_size)))
    y_train.append(label)
    i += 1

 32%|███▏      | 3230/10222 [00:09<00:21, 325.46it/s]/usr/local/lib/python3.6/dist-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
100%|██████████| 10222/10222 [00:31<00:00, 326.11it/s]


In [11]:
for f in tqdm(dog_test['id'].values):
    img = cv2.imread('dog-breed-classification-kaggle/test/test/{}.jpg'.format(f))
    x_test.append(cv2.resize(img, (image_size, image_size)))

 32%|███▏      | 3302/10357 [00:09<00:21, 332.19it/s]/usr/local/lib/python3.6/dist-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
100%|██████████| 10357/10357 [00:31<00:00, 327.14it/s]


In [5]:
import numpy as np
y_train_raw = np.array(y_train, np.uint8)
x_train_raw = np.array(x_train, np.float32) / 255.
x_test  = np.array(x_test, np.float32) / 255.

NameError: ignored

In [2]:
print(x_train_raw.shape)
print(y_train_raw.shape)
print(x_test.shape)

NameError: ignored

In [3]:
num_class = y_train_raw.shape[1]

NameError: ignored

In [0]:
X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size = 0.3, random_state = 1)

In [4]:
base_model = VGG19(#weights='imagenet',
    weights = None, include_top=False, input_shape=(image_size, image_size, 3))

x = base_model.output
x = Flatten()(x)
predictions = Dense(num_class, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False

model.compile(loss = 'categorical_crossentropy', 
              optimizer = 'adam', 
              metrics = ['accuracy'])

callbacks_list = [
    keras.callbacks.EarlyStopping(monitor = 'val_accuracy', patience = 3, verbose = 1)
                 ]


NameError: ignored

In [3]:
model.summary()
model.save_weights(dog_pred1.h5)
model.fit(X_train, Y_train, epochs = 1, validation_data = (X_valid, Y_valid), verbose = 1)

NameError: ignored

In [2]:
prediction = model.predict(x_test)

NameError: ignored

In [1]:
prediction == prediction.max(axis = 1, keepdims=True)
prediction = (prediction == prediction.max(axis = 1, keepdims = True)).astype(int)

NameError: ignored

In [0]:
sub = pd.DataFrame(preds)
col_names = one_hot.columns.values
sub.columns = col_names
sub.insert(0, 'id', df_test['id'])
sub.head(5)